## Analysis of a Stabilization graph
Based on J.S.-Y. Chao *et al.*, *J. Chem. Phys.* **93**, 1125 (1990)

See also M. F. Falcetta *et al.*, *J. Phys. Chem. A* **118**, 7489 (2014) 

Analytic continuation of two roots using a generalized Pade approximant and input data from a crossing.

In [6]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5

In [7]:
import sys
sys.path.append('../../Python_libs')
import gen_pade as gpa
import stabtools as st

In [8]:
def gpa_ana(zs, E1s, E2s, Pade_order):
    """
    GPA analysis in one function for gpa_scatter 
    Always, always run gpa_analysis first. 
    Not a black box like 2-by-2 model Hamiltonian.
    zs: The input alphas or Ls 
    E1s: one branch of the crossing, E1s(zs)
    E2s: the other branch, E2s(zs)
    Pade_order: (nP, nQ, nR) = orders of the GPA
    """
    zzs = list(zs) + list(zs)
    EEs = list(E1s) + list(E2s)
    nP, nQ, nR = Pade_order
    PQR = gpa.genpade2_via_lstsq(nP, nQ, nR, zzs, EEs, rcond=1e-14)
    
    # area for starting Newton searches
    zc=(zs[-1]+zs[0])/2
    half_d=(zs[-1]-zs[0])/2
    re1=zc-2*half_d
    re2=zc+2*half_d
    im1=-4.2*half_d
    im2=-0.2*half_d
    npts = 10
    Res=np.linspace(re1,re2,npts) 
    Ims=np.linspace(im1,im2,npts)

    Econv = []
    for Re in Res:
        for Im in Ims:
            Z = Re + 1j*Im
            conv, zstar, Estar, der = gpa.GPA_NewtonRaphson(Z, PQR, max_step=10, verbose=False)
            if conv:
                zr, zi = zstar.real, zstar.imag
                Er, Ei = Estar.real, Estar.imag
                accept_z = (zr>re1 and zr<re2 and zi>im1 and zi<im2)
                accept_E = (Er>0 and Ei<0)
                if accept_z and accept_E:
                    Econv.append(np.round(Estar,6))

    return np.unique(Econv)

In [28]:
Angs2Bohr=1.8897259886
au2eV=27.211386027
au2cm=219474.63068
#
#dfin=pd.read_csv("DVR_stab_plot.csv", delim_whitespace=False)
dfin=pd.read_csv("GTO_UN_stab_plot.csv", delim_whitespace=False)
#dfin=pd.read_csv("GTO_DZ_stab_plot.csv", delim_whitespace=False)
#dfin=pd.read_csv("GTO_TZ_stab_plot.csv", delim_whitespace=False)
#
#df=pd.read_csv("/home/thomas/Current_Work/Caroline/Glyoxal/Stab/E_geo0.csv", delim_whitespace=True)
#df=pd.read_csv("/home/thomas/Current_Work/Caroline/Glyoxal/Stab/E_geo-1.csv", delim_whitespace=True)
#df=pd.read_csv("/home/thomas/Current_Work/Caroline/Glyoxal/Stab/Z_geo0_cleaned.csv")
#df=pd.read_csv("/home/thomas/Current_Work/Caroline/Glyoxal/Stab/Z_geo-1_cleaned.csv")

dfin.tail()

,L,z,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,E11,E12,E13,E14
155,0.733285,1.859748,-7.170457,0.910726,2.907185,3.752651,7.203296,11.997494,19.096021,31.143320,58.435500,121.236694,255.397202,538.359713,1167.893045,3239.400811
156,0.731309,1.869811,-7.170457,0.918185,2.915043,3.769399,7.235874,12.043933,19.164357,31.230128,58.526094,121.322791,255.485518,538.489387,1168.320207,3248.963838
157,0.729349,1.879874,-7.170457,0.925668,2.922651,3.786366,7.268337,12.090294,19.232558,31.316828,58.616674,121.408897,255.573730,538.618317,1168.742700,3258.529694
158,0.727405,1.889937,-7.170457,0.933177,2.930017,3.803542,7.300687,12.136576,19.300625,31.403420,58.707240,121.495012,255.661839,538.746517,1169.160601,3268.098428
159,0.725476,1.900000,-7.170458,0.940711,2.937151,3.820920,7.332924,12.182781,19.368558,31.489905,58.797791,121.581136,255.749848,538.874000,1169.573982,3277.670089


In [29]:
alpha=dfin.columns[1]
E_labels=dfin.columns[2:]
all_zs=dfin[alpha].values
zmin, zmax = all_zs[0], all_zs[-1]
if zmax < zmin:
    zmin, zmax = zmax, zmin
all_zs=(all_zs-zmin)/(zmax-zmin)
if True:
    plt.cla()
    for E in E_labels:
        plt.plot(all_zs, dfin[E].values, 'o-')
    plt.ylim(0,6)
    plt.show()

Set the following parameters after inspecting the stabilization plot:
* lroot: the lower root; the 2nd root is lroot+1
* curvature_drop-off: the crossing is defined by the minimum and maximum curvature positions; the  selection range is determined by the drop off of the abs(curvature) 

In [30]:
lroot=3
curvature_drop_off=0.3
E_lw=dfin[E_labels[lroot]].values
E_up=dfin[E_labels[lroot+1]].values
success, zc, zs, E1s, E2s = st.crossing(all_zs, E_lw, E_up, select=curvature_drop_off)
N = len(zs)
if not success or N < 2:
    print('No crossing found. Try debug box in GPA-analysis.ipynb')
else:
    print('%d%% of the crossing at %f' % ((1-curvature_drop_off)*100, zc))
    print('%d points on each curve' % (N))
    plt.cla()
    plt.plot(all_zs, E_lw, all_zs, E_up)
    plt.plot(zs, E1s, 'o', zs, E2s, 'o')
    plt.ylabel('roots '+str(lroot)+' and '+str(lroot+1))
    plt.show()

70% of the crossing at 0.352201
33 points on each curve


In [31]:
by_hand=False
if by_hand:
    z_min=0.105
    z_max=0.35
    jmin=np.argmin(np.abs(zs-z_min))
    jmax=np.argmin(np.abs(zs-z_max))+1
    if jmax > len(zs):
        jmax = -1
    alps=zs[jmin:jmax]
    E1s=E_lw[jmin:jmax]
    E2s=E_up[jmin:jmax]
    print(f'{len(alps)} points on each curve')
    plt.cla()
    plt.plot(zs, E_lw, zs, E_up)
    plt.plot(alps, E1s, 'o', alps, E2s, 'o')
    plt.ylabel('roots '+str(lroot)+' and '+str(lroot+1))
    plt.show()    

In [32]:
if True:
    # Test with the complete data set
    E_conv = gpa_ana(zs, E1s, E2s, (2,3,4))
    nE = len(E_conv)
    print(f'{nE} unique stationary energies')
    for E in E_conv:
        print(f'{E.real}  {E.imag}')

4 unique stationary energies
3.00541  -1.723169
3.109589  -0.128212
3.174855  -0.366495
3.208242  -0.202086


In [33]:
nP, nQ, nR = 2, 3, 4
Pade_order=(nP, nQ, nR)
min_data=(nP+1)+(nQ+1)+(nR+1) # well-defined GPA 
print(f'min(data)={min_data}')

ils=[]
irs=[]
Ers=[]
Eis=[]

j_mid=np.argmin(abs(zs-zc))
for ilft in range(j_mid-1):
    for irht in range(N,j_mid+1,-1):
        if irht - ilft < min_data:
            continue
        zsel, E1sel, E2sel = zs[ilft:irht], E1s[ilft:irht], E2s[ilft:irht]
        E_conv = gpa_ana(zsel, E1sel, E2sel, Pade_order)
        for Eres in E_conv:
            ils.append(ilft)
            irs.append(N-irht)
            Ers.append(Eres.real)
            Eis.append(Eres.imag)
            print(f'{ilft:4d}  {N-irht:4d}   {Eres.real:8f} {Eres.imag:8f}')

min(data)=12
   0     0   3.005410 -1.723169
   0     0   3.109589 -0.128212
   0     0   3.174855 -0.366495
   0     0   3.208242 -0.202086
   0     1   3.076365 -0.125653
   0     1   3.175298 -0.264217
   0     1   3.203972 -0.229265
   0     1   3.893101 -0.903079
   0     2   3.160636 -0.217579
   0     3   3.159421 -0.202398
   0     4   3.160873 -0.195049
   0     5   3.162435 -0.190812
   0     6   3.163810 -0.188077
   0     7   3.164997 -0.186187
   0     7   3.181121 -0.066729
   0     8   3.166029 -0.184821
   0     9   3.166937 -0.183810
   0    10   3.167743 -0.183052
   0    11   3.168465 -0.182485
   0    12   3.169112 -0.182069
   0    13   3.169695 -0.181779
   0    14   3.170216 -0.181596
   1     0   2.585011 -2.250767
   1     0   3.127867 -0.129651
   1     0   3.203548 -0.189990
   1     0   4.302858 -1.827672
   1     1   3.110789 -0.128447
   1     1   3.180606 -0.367692
   1     1   3.206163 -0.201109
   1     1   4.080013 -1.382808
   1     2   3.076428 -0.12

In [34]:
dic={'left':ils, 'right':irs, 'Er':Ers, 'Ei':Eis}
df=pd.DataFrame(dic)
df.plot.scatter(x='Er', y='Ei')
df.describe()

,left,right,Er,Ei
count,231.000000,231.000000,231.000000,231.000000
mean,4.653680,4.324675,3.228409,-0.311509
std,3.101398,3.322289,0.276383,0.450354
min,0.000000,0.000000,2.585011,-2.337153
25%,2.000000,1.500000,3.159382,-0.200342
50%,5.000000,4.000000,3.167999,-0.182089
75%,7.000000,7.000000,3.192927,-0.141915
max,11.000000,14.000000,4.620005,-0.036239


Primitive cluster analysis 

In [36]:
mean_Er=df.describe()['Er']['mean']
mean_Ei=df.describe()['Ei']['mean']
std_Er =df.describe()['Er']['std']
std_Ei =df.describe()['Er']['std']
s=1.0
pattern=( (abs(df['Er']-mean_Er) < s*std_Er) & (abs(df['Ei']-mean_Ei) < s*std_Ei) )
df2=df[pattern]
df2.plot.scatter(x='Er', y='Ei')
df2.describe()

,left,right,Er,Ei
count,208.000000,208.000000,208.000000,208.000000
mean,4.812500,4.475962,3.165256,-0.171558
std,3.155922,3.376446,0.035264,0.043467
min,0.000000,0.000000,2.976909,-0.367692
25%,2.000000,2.000000,3.159137,-0.189501
50%,5.000000,4.000000,3.166492,-0.178352
75%,7.000000,7.000000,3.190635,-0.139591
max,11.000000,14.000000,3.229471,-0.036239


In [37]:
xs=df2.Er.values
ys=df2.Ei.values

In [42]:
#
#  long vs short datasets
#
j_left=df2.left.values*1.0
j_right=df2.right.values*1.0
sum_pts=j_left + j_right
#sum_pts=left + right
plt.cla()
plt.scatter(xs, ys, marker='.', c=sum_pts, cmap='viridis')
plt.colorbar()
plt.show()

In [43]:
#
#  symmetric vs unsymmetric datasets
#
diff_pts=abs(j_left - j_right)
plt.cla()
plt.scatter(xs, ys, marker='.', c=diff_pts, cmap='viridis')
plt.colorbar()
plt.show()

In [41]:
# both

fig, axs = plt.subplots(1, 2, sharex=True, sharey=True)

fig.set_figwidth(6.4)
fig.set_figheight(3.2)

#axs[0].set_xticks([3.15, 3.18])

axs[0].scatter(xs, ys, marker='.', c=sum_pts, cmap='viridis')
axs[1].scatter(xs, ys, marker='.', c=diff_pts, cmap='viridis')
axs[0].plot(3.17296,-0.160848,'k+',markersize=10)
axs[1].plot(3.17296,-0.160848,'k+',markersize=10)
axs[0].set_xlabel("$E_r$ [eV]", fontsize=12)
axs[1].set_xlabel("$E_r$ [eV]", fontsize=12)
axs[0].set_ylabel("$E_i$ [eV]", fontsize=12)

plt.tight_layout()
plt.show()